# DATA2001 Practical Assignment: Greater Sydney Analysis
SID: 520108420, 520360707

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon
from shapely import wkt
from geoalchemy2 import Geometry, WKTElement
import matplotlib.pyplot as plt

In [2]:
# Loading in data
businesses = pd.read_csv('Businesses.csv')
income = pd.read_csv('Income.csv')
polling = pd.read_csv('PollingPlaces2019.csv')
population = pd.read_csv('Population.csv')
stops = pd.read_csv('Stops.txt', sep=',')
school_p = gpd.read_file('catchments/catchments_primary.shp')
school_s = gpd.read_file('catchments/catchments_secondary.shp')
sa2 = gpd.read_file('SA2_2021_AUST_SHP_GDA2020/SA2_2021_AUST_GDA2020.shp')

### Data cleaning

In [3]:
# Removing unnecessary columns
# Businesses
businesses = businesses.drop(columns=['industry_code','0_to_50k_businesses','50k_to_200k_businesses','200k_to_2m_businesses','2m_to_5m_businesses','5m_to_10m_businesses','10m_or_more_businesses'])
# Income
income = income.drop(columns=['earners','median_age','mean_income'])
income = income.loc[income['median_income']!= 'np']
# Population - removing columns and combining age groups to form young people category
young_people = ['0-4_people','5-9_people','10-14_people','15-19_people']
population['young_people']=population[young_people].sum(axis=1)
population = population.drop(columns=['0-4_people','5-9_people','10-14_people','15-19_people','20-24_people','25-29_people','30-34_people','35-39_people','40-44_people','45-49_people','50-54_people','55-59_people','60-64_people','65-69_people','70-74_people','75-79_people','80-84_people','85-and-over_people'])
# Polling
polling = polling.drop(columns=['premises_name','FID','state','division_id','division_name','polling_place_type_id','premises_address_1','premises_address_2','premises_address_3','premises_suburb','premises_state_abbreviation','premises_post_code'])
polling = polling.dropna()
# Stops
stops = stops.drop(columns=['stop_code','location_type','parent_station','wheelchair_boarding','platform_code'])
# School - removing columns and combining primary and secondary schools into 1 dataframe
school = pd.concat([school_p,school_s])
school = school.drop(columns=['CATCH_TYPE','ADD_DATE','KINDERGART','YEAR1','YEAR2','YEAR3','YEAR4','YEAR5','YEAR6','YEAR7','YEAR8','YEAR9','YEAR10','YEAR11','YEAR12','PRIORITY'])
# SA2
sa2 = sa2.drop(columns=['CHG_FLAG21','CHG_LBL21','SA3_CODE21','SA3_NAME21','SA4_CODE21','SA4_NAME21','GCC_CODE21','STE_CODE21','STE_NAME21','AUS_CODE21','AUS_NAME21','AREASQKM21','LOCI_URI21'])

# Filtering
# Businesses
businesses = businesses.loc[businesses['industry_name'].isin(['Retail Trade','Health Care and Social Assistance'])]
# SA2
sa2 = sa2.loc[sa2['GCC_NAME21'] == 'Greater Sydney']
sa2 = sa2.drop(columns=['GCC_NAME21'])

# Store geographical point 
stops['the_geom'] = gpd.points_from_xy(stops.stop_lon, stops.stop_lat)
stops = stops.drop(columns=['stop_lat', 'stop_lon'])
polling['the_geom'] = gpd.points_from_xy(polling.longitude, polling.latitude)
polling = polling.drop(columns=['latitude', 'longitude'])


# Convert column names to lower case
businesses.columns = businesses.columns.str.lower()
income.columns = income.columns.str.lower()
polling.columns = polling.columns.str.lower()
population.columns = population.columns.str.lower()
stops.columns = stops.columns.str.lower()
school.columns = school.columns.str.lower()
sa2.columns = sa2.columns.str.lower()


#### Check SRIDs

In [4]:
# school
srid = school.crs.to_epsg()
print('SRID:', srid)

SRID: 4283


In [5]:
# sa2
srid = sa2.crs.to_epsg()
print('SRID:', srid)

SRID: 7844


#### SRID transformations

In [6]:
# stops
srid = 4326
stops['the_geom'] = stops['the_geom'].apply(lambda x: WKTElement(x, srid=srid))

In [7]:
# Polling
polling['the_geom'] = polling['the_geom'].apply(lambda x: WKTElement(x, srid=srid))

In [8]:
# school
target_crs = 'EPSG:4326'  # For example, WGS84

# Convert the CRS of the DataFrame
school = school.to_crs(target_crs)

# Convert the individual polygons to MULTIPOLYGON
school['geometry'] = school['geometry'].apply(lambda x: MultiPolygon([x]) if isinstance(x, Polygon) else x)

In [9]:
# sa2
target_crs = 'EPSG:4326'  # For example, WGS84

# Convert the CRS of the DataFrame
sa2 = sa2.to_crs(target_crs)

# Convert the individual polygons to MULTIPOLYGON
sa2['geometry'] = sa2['geometry'].apply(lambda x: MultiPolygon([x]) if isinstance(x, Polygon) else x)

In [10]:
from sqlalchemy import create_engine
import psycopg2
import psycopg2.extras
import json
import os

credentials = "Credentials.json"

def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host       = db_conn_dict['host']
        db_user    = db_conn_dict['user']
        db_pw      = db_conn_dict['password']
        default_db = db_conn_dict['user']
        try:
            db = create_engine('postgresql+psycopg2://'+db_user+':'+db_pw+'@'+host+'/'+default_db, echo=False)
            conn = db.connect()
            print('Connected successfully.')
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        return db,conn

def query(conn, sqlcmd, args=None, df=True):
    result = pd.DataFrame() if df else None
    try:
        if df:
            result = pd.read_sql_query(sqlcmd, conn, params=args)
        else:
            result = conn.execute(sqlcmd, args).fetchall()
            result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered: ", e, sep='\n')
    return result

db, conn = pgconnect(credentials)

Connected successfully.


In [11]:
conn.execute("""
create schema if not exists public;
set search_path to public;
""")

In [12]:
conn.execute("""
DROP TABLE IF EXISTS businesses;
CREATE TABLE businesses (
    id SERIAL PRIMARY KEY,
    industry_name VARCHAR(100), 
    sa2_code INTEGER, 
    sa2_name VARCHAR(100),
    total_businesses INTEGER
);"""
)

In [13]:
conn.execute("""
DROP TABLE IF EXISTS polling;
CREATE TABLE polling (
    polling_place_id INTEGER PRIMARY KEY,
    polling_place_name VARCHAR(100),
    the_geom public.GEOMETRY(POINT,4326)
);"""
)

In [14]:
conn.execute("""
DROP TABLE IF EXISTS income;
CREATE TABLE income (
    sa2_code INTEGER PRIMARY KEY,
    sa2_name VARCHAR(100),
    median_income INTEGER
);"""
)

In [15]:
conn.execute("""
DROP TABLE IF EXISTS population;
CREATE TABLE population (
    sa2_code INTEGER PRIMARY KEY,
    sa2_name VARCHAR(100),
    total_people INTEGER,
    young_people INTEGER
);"""
)

In [16]:
conn.execute("""
DROP TABLE IF EXISTS stops;
CREATE TABLE stops (
    stop_id VARCHAR(100) PRIMARY KEY,
    stop_name VARCHAR(100),
    the_geom public.GEOMETRY(POINT,4326)
);"""
)

In [17]:
conn.execute("""
DROP TABLE IF EXISTS school;
CREATE TABLE school (
    use_id INTEGER PRIMARY KEY,
    use_desc VARCHAR(100),
    geometry public.GEOMETRY(MULTIPOLYGON,4326)
);"""
)

In [18]:
conn.execute("""
DROP TABLE IF EXISTS sa2;
CREATE TABLE sa2 (
    sa2_code21 INTEGER PRIMARY KEY,
    sa2_name21 VARCHAR(100),
    geometry public.GEOMETRY(MULTIPOLYGON,4326)
);"""
)

### Ingestion

In [19]:
businesses.to_sql('businesses', conn, if_exists='append', index=False)
income.to_sql('income', conn, if_exists='append', index=False)
population.to_sql('population', conn, if_exists='append', index=False)
sa2.to_postgis(name='sa2', con=conn, if_exists='replace', index=False, schema='public', dtype={'geometry': 'MULTIPOLYGON'})
school.to_postgis(name='school', con=conn, if_exists='replace', index=False, schema='public', dtype={'geometry': 'MULTIPOLYGON'})
polling.to_sql('polling', conn, if_exists='append', index=False, dtype={'the_geom':Geometry('POINT',srid)})
stops.to_sql('stops', conn, if_exists='append', index=False, dtype={'the_geom':Geometry('POINT',srid)})

718

In [31]:
conn.execute("""
DROP TABLE IF EXISTS sa2_nonspatial;
CREATE TABLE sa2_nonspatial AS
SELECT s.sa2_code21, s.sa2_name21, b.industry_name, b.total_businesses, p.young_people, p.total_people, s.geometry
FROM businesses b
JOIN sa2 s
ON b.sa2_code = s.sa2_code21::integer
JOIN population p
ON p.sa2_code = s.sa2_code21::integer
where p.total_people >= 100
;"""
)

In [33]:
conn.execute("""
DROP TABLE IF EXISTS sa2_spatial;
CREATE TABLE sa2_spatial AS 
SELECT sa2.sa2_code21, sa2.sa2_name21,
       COUNT(DISTINCT stops.stop_id) AS stop_count,
       COUNT(DISTINCT polling.polling_place_id) AS polling_place_count,
       COUNT(DISTINCT school.use_id) AS school_catchment_count
FROM school
LEFT JOIN sa2 ON ST_INTERSECTS(sa2.geometry, school.geometry)
LEFT JOIN stops ON ST_Intersects(sa2.geometry, stops.the_geom)
LEFT JOIN polling ON ST_Intersects(sa2.geometry, polling.the_geom)
GROUP BY sa2.sa2_code21, sa2.sa2_name21;
;"""
)

In [40]:
conn.execute("""
DROP TABLE IF EXISTS all_data;
CREATE TABLE all_data AS 
SELECT s.*, n.industry_name, n.total_businesses, n.young_people, n.total_people, n.geometry
FROM sa2_spatial s
JOIN sa2_nonspatial n
ON s.sa2_code21 = n.sa2_code21;
;"""
)

In [45]:
query(conn, "select * from all_data")

,sa2_code21,sa2_name21,stop_count,polling_place_count,school_catchment_count,industry_name,total_businesses,young_people,total_people,geometry
0,102011028,Avoca Beach - Copacabana,127,2,6,Health Care and Social Assistance,74,2121,7530,0106000020E6100000010000000103000000010000005E...
1,102011028,Avoca Beach - Copacabana,127,2,6,Retail Trade,45,2121,7530,0106000020E6100000010000000103000000010000005E...
2,102011029,Box Head - MacMasters Beach,215,2,7,Health Care and Social Assistance,55,2471,11052,0106000020E61000000100000001030000000100000010...
3,102011029,Box Head - MacMasters Beach,215,2,7,Retail Trade,50,2471,11052,0106000020E61000000100000001030000000100000010...
4,102011030,Calga - Kulnura,172,7,26,Health Care and Social Assistance,43,961,4748,0106000020E61000000200000001030000000100000085...
...,...,...,...,...,...,...,...,...,...,...
715,128021607,Engadine,221,7,10,Retail Trade,68,5118,17379,0106000020E6100000010000000103000000010000008E...
716,128021608,Loftus - Yarrawarrah,144,2,9,Health Care and Social Assistance,22,2073,7354,0106000020E610000001000000010300000001000000A1...
717,128021608,Loftus - Yarrawarrah,144,2,9,Retail Trade,29,2073,7354,0106000020E610000001000000010300000001000000A1...
718,128021609,Woronora Heights,42,1,8,Health Care and Social Assistance,15,965,3551,0106000020E610000001000000010300000001000000F0...


### Compute z-scores

### Normalise z-scores

### Compute score

### Extend score by additional datasets

### Visualise score

### Determine correlation between score and median income

In [2]:
#conn.close()
#db.dispose()